# cropping


In [1]:
import cv2
import math
import matplotlib.pyplot as plt
import mediapipe as mp
import random
import os
from IPython.display import display, Image
from PIL import Image
import numpy as np


class mpFace:
    import mediapipe as mp
    import cv2
      
    def __init__(self,width=640,height=480):
        self.findFace = self.mp.solutions.face_detection.FaceDetection()
        self.faceMesh = self.mp.solutions.face_mesh.FaceMesh(False,3,True,0.5,0.5)
        #(staticFrame,number of faces,True for extra iris landmarks,trackingParameter,findingParameter)
        self.width = width
        self.height = height
        
        
    def faceBox(self,frame):#face bounding box
        frameRGB = self.cv2.cvtColor(frame,self.cv2.COLOR_BGR2RGB)
        results = self.findFace.process(frameRGB)
        myFaces = []
        if results.detections != None:
            for face in results.detections:
                bBox = face.location_data.relative_bounding_box
                topLeft = (int(bBox.xmin*self.width),int(bBox.ymin*self.height))
                bottomRight = (int((bBox.xmin+bBox.width)*self.width),int((bBox.ymin+bBox.height)*self.height))
                myFaces.append((topLeft,bottomRight))
        return myFaces  
    
    def faceLandmarks(self,frame):#Full Face Landmarks
        frameRGB = self.cv2.cvtColor(frame,self.cv2.COLOR_BGR2RGB)# Converts BGR (OpenCV default) color format to RGB format required by MediaPipe.
        results = self.faceMesh.process(frameRGB) #Processes the RGB frame using the faceMesh model to detect facial landmarks
        myFacesLandmarks=[] #Initializes an empty list to store landmarks for each detected face.
        myFaceLandmarks=[]  #Initializes an empty list to store landmarks for a single face.
    
        if results.multi_face_landmarks != None:
            for faceLandmarks in results.multi_face_landmarks:
                for lm in faceLandmarks.landmark:
                    myFaceLandmarks.append((int(lm.x*self.width),int(lm.y*self.height),int(lm.z*self.width)))

                myFacesLandmarks.append(myFaceLandmarks)            
        return myFacesLandmarks

    def faceLandmarksSimplified(self,frame):#essential face landmarks(left eyebrow,righteyebrow,left eye,right eye,inner lips,outer lips,face outline,left iris and right iris)
        frameRGB = self.cv2.cvtColor(frame,self.cv2.COLOR_BGR2RGB)
        results = self.faceMesh.process(frameRGB)
        #sequenced indexes of required landmarks from original landmarks
        points = [70,63,105,66,107,55,65,52,53,46,300,293,334,296,336,285,295,282,283,276,33,246,161,160,159,158,157,173,133,155,154,153,145,144,163,7,263,466,388,387,386,385,384,398,362,382,381,380,374,373,390,249,78,191,80,81,82,13,312,311,310,415,308,324,318,402,317,14,87,178,88,95,61,185,40,39,37,0,267,269,270,409,291,375,321,405,314,17,84,181,91,146,10,338,297,332,284,251,389,356,454,323,361,288,397,365,379,378,400,377,152,148,176,149,150,136,172,58,132,93,234,127,162,21,54,103,67,109,468,469,470,471,472,473,474,475,476,477,64,4,294]
        myFaceLandmarksSimplified=[]
        myFacesLandmarksSimplified=[]
        myFaceLandmarksRearranged = [0]*len(points)
        myIndex=[]
    
        if results.multi_face_landmarks != None:
            for faceLandmarks in results.multi_face_landmarks:
                for lm,indx in zip(faceLandmarks.landmark,range(len(faceLandmarks.landmark))):
                    if indx in points:#only collect required landmarks
                        myFaceLandmarksSimplified.append((int(lm.x*self.width),int(lm.y*self.height),int(lm.z*self.width)))
                        myIndex.append(points.index(indx))#for rearranging the points, collect sequenced index
                    for i,indx in zip(range(len(points)),myIndex):
                        myFaceLandmarksRearranged[indx] = myFaceLandmarksSimplified[i]#rearranging according to sequenced index

                myFacesLandmarksSimplified.append(myFaceLandmarksRearranged)          
        return myFacesLandmarksSimplified
    

radius = 2
#red = (0,0,255)
green = (0,255,0)
blue = (255,0,0)
yellow = (0,255,255)


# Define transparent colors
transparent_red = (0, 0, 255, 0)    # Transparent red
transparent_green = (0, 255, 0, 0)  # Transparent green

#For connecting range of dots
def connectPoints(indx1,indx2):
    for i in range(indx1,indx2):
        if i==(indx2-1):
            cv2.line(frame,(face[i][0],face[i][1]),(face[indx1][0],face[indx1][1]),transparent_green,1)
            break
        cv2.line(frame,(face[i][0],face[i][1]),(face[i+1][0],face[i+1][1]),transparent_green,1)
        
#Finding length between two points
def findRadius(pt1,pt2):
    x1,y1 = (pt1[0],pt1[1])
    x2,y2 = (pt2[0],pt2[1])
    radius = math.sqrt(((y2-y1)*(y2-y1))+((x2-x1)*(x2-x1)))
    return radius

#boundary box code 
def findBoundingBox(lip_landmarks, margin=30):
    min_x = min(lip_landmarks, key=lambda x: x[0])[0] - margin
    max_x = max(lip_landmarks, key=lambda x: x[0])[0] + margin
    min_y = min(lip_landmarks, key=lambda x: x[1])[1] - margin
    max_y = max(lip_landmarks, key=lambda x: x[1])[1] + margin
    return (min_x, min_y, max_x, max_y)

def lips_detect(image_path):
    # Load an image from the filesystem
    original_frame = cv2.imread(image_path)

    # Resize the image to a fixed size (640x480)
    fixed_width = 640
    fixed_height = 480
    frame = cv2.resize(original_frame, (fixed_width, fixed_height))
    
    # Initialize the variable before the loop
    outer_lip_cropped = None

    # Perform face detection and lip landmarks
    faceLm = mpFace()
    faces = faceLm.faceLandmarksSimplified(frame)
    
    for face in faces:
        outer_lip_landmarks = face[72:92]
        outer_lip_box = findBoundingBox(outer_lip_landmarks, margin=10)

        # Crop the outer lip region
        outer_lip_cropped = frame[outer_lip_box[1]:outer_lip_box[3], outer_lip_box[0]:outer_lip_box[2]]

    return frame, outer_lip_cropped

def image_generator(folder_path, file_extensions):
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(file_extensions):
            yield os.path.join(folder_path, filename)

def process_images(image_paths):
    for image_path in image_paths:
        output_frame, cropped_lips = lips_detect(image_path)  # Call lips_detect function
        
        if cropped_lips is not None:  # Check if lips were detected
            # Display the cropped image using PIL
            display(Image.fromarray(cv2.cvtColor(cropped_lips, cv2.COLOR_BGR2RGB)))
            
            
import os
import random
import cv2
from PIL import Image
from IPython.display import display

# Define the folder path and output folder
folder_path = r'C:\Users\suraj\Desktop\happy sad netural\without_padding\train\happy\9'
output_folder = r'C:\Users\suraj\Desktop\happy sad netural\without_padding\train\happy\9\1'

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
    

lst = os.listdir(folder_path) # input directory path
file_number = len(lst)

print('file number : ',file_number)

# Iterate through images, detect lips, crop, and display
max_images = file_number  # Limit the number of images processed
image_count = 0

for filename in os.listdir(folder_path):
    if image_count >= max_images:
        break

    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):
        image_count += 1
        image_path = os.path.join(folder_path, filename)
        output_frame, cropped_lips = lips_detect(image_path)  # Call lips_detect function
        
        if cropped_lips is not None:  # Check if lips were detected
            
            # Save the cropped image to the output folder
            output_filename = os.path.splitext(filename)[0] + '_cropped.jpg'
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, cropped_lips)
            
            # Display the cropped image using PIL
            #display(Image.fromarray(cv2.cvtColor(cropped_lips, cv2.COLOR_BGR2RGB)))
            print('Converted : ', image_count)
          
            
            # Save the cropped image to the output folder
            output_filename = os.path.splitext(filename)[0] + '_cropped.jpg'
            output_path = os.path.join(output_folder, output_filename)
            cv2.imwrite(output_path, cropped_lips)
        
        
lst = os.listdir(output_folder) # output directory path
output_file_number = len(lst)


print(" Input file number : ", image_count)
print("output file number : ", output_file_number)
print("Image cropping and displaying completed.")

    

file number :  21
Converted :  1
Converted :  2
Converted :  3
Converted :  4
Converted :  5
Converted :  6
Converted :  7
Converted :  8
Converted :  9
Converted :  10
Converted :  11
Converted :  12
Converted :  13
Converted :  14
Converted :  15
Converted :  16
Converted :  17
Converted :  18
Converted :  19
Converted :  20
 Input file number :  20
output file number :  20
Image cropping and displaying completed.


# letter box 

In [3]:
from PIL import Image
import os

def letterbox_images_in_folder(input_folder, output_folder, new_width, new_height, max_images=None):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    lst = os.listdir(input_folder)  # Input directory path
    file_number = len(lst)
    print('file number : ', file_number)

    # Iterate through images, resize using letterbox, and save
    image_count = 0

    for filename in os.listdir(input_folder):
        if max_images is not None and image_count >= max_images:
            break

        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp')):  # Adjust the file extension if needed
            image_count += 1
            # Load the original image
            original_image = Image.open(os.path.join(input_folder, filename))

            # Calculate the scaling factors for resizing
            width_ratio = new_width / original_image.width
            height_ratio = new_height / original_image.height
            scaling_factor = min(width_ratio, height_ratio)

            # Calculate the new dimensions after resizing
            target_width = int(original_image.width * scaling_factor)
            target_height = int(original_image.height * scaling_factor)

            # Resize using the Letterbox technique
            #resized_image = original_image.resize((target_width, target_height), Image.ANTIALIAS)
            resized_image = original_image.resize((target_width, target_height), Image.Resampling.LANCZOS)
            letterboxed_image = Image.new("RGB", (new_width, new_height), (255, 215, 174))

            # Calculate the position to paste the resized image
            paste_x = (new_width - target_width) // 2
            paste_y = (new_height - target_height) // 2

            # Paste the resized image onto the letterboxed image
            letterboxed_image.paste(resized_image, (paste_x, paste_y))

            letterboxed_filename = os.path.splitext(filename)[0] + "_letterboxed.jpg"
            output_path = os.path.join(output_folder, letterboxed_filename)
            letterboxed_image.save(output_path)

            print('Converted : ', image_count)

    print("Letterboxing and saving complete.")

# Define your input and output paths, and other parameters
input_folder = r"C:\Users\suraj\Desktop\happy sad netural\without_padding\train\happy\9\1"
output_folder = r"C:\Users\suraj\Desktop\happy sad netural\without_padding\train\happy\9\1\letterboxed"
new_width = 224
new_height = 224
max_images = None  # Set the maximum number of images to process, or leave it as None to process all

# Call the function to letterbox and save images
letterbox_images_in_folder(input_folder, output_folder, new_width, new_height, max_images)


file number :  21
Converted :  1
Converted :  2
Converted :  3
Converted :  4
Converted :  5
Converted :  6
Converted :  7
Converted :  8
Converted :  9
Converted :  10
Converted :  11
Converted :  12
Converted :  13
Converted :  14
Converted :  15
Converted :  16
Converted :  17
Converted :  18
Converted :  19
Converted :  20
Letterboxing and saving complete.
